# Activity 8: Re-training a model dynamically
In this activity, we re-train our model every time new data is available.

First, we start by importing `cryptonic`. Cryptonic is a simple software application developed for this course that implements all the steps up to this section using Python classes and modules. Consider Cryptonic a template on how you could develop similar applications.

In [ ]:
from cryptonic import Model
from cryptonic import CoinMarketCap
from cryptonic.models import normalizations

from tqdm import tqdm_notebook

import pandas as pd

#%autosave 5
%matplotlib inline

## A seperate notebook is dedicated for downloading data from kaggle and preprocessing the data suitable for training the model


The data contains practically the same variables from our earlier dataset. However, much of the data comes from an earlier period. Recent Bitcoin prices have gained a lot of volatility if compared to the prices of a few years ago. Before using this data in our model, let's make sure to filter it to dates after January 1, 2017.

In [10]:
#
#  Using the Pandas API, filter the dataframe
#  for observations from 2017 only. 
# 
#  Hint: use the `date` variable.
#
model_data = pd.read_csv("/home/praveen/Desktop/Projects/technocolab/own/historic_data.csv")
model_data = model_data[model_data['date'] >= '2017-01-01']
model_data = model_data[model_data['date'] <= '2018-01-01']
model_data.tail()

,date,close,iso_week,volume
2189,2017-12-28,14496.923236,2017-58,11.562764
2190,2017-12-29,14428.620743,2017-59,9.542329
2191,2017-12-30,13559.031569,2017-59,15.210698
2192,2017-12-31,13106.492139,2017-59,8.894869
2193,2018-01-01,13548.910740,2018-01,5.323438


## The `Model()` Class
We have also created the class `Model()` which compiles all the code we have written so far. We will use that class to build, train, and evaluate our model.

In [11]:
M = Model(data=model_data,
          variable='close',
          predicted_period_size=7)

In [12]:
M.build()

In [13]:
M.train(epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 587us/step - loss: 0.0096
Epoch 2/100
1/1 [==============================] - 0s 532us/step - loss: 0.0079
Epoch 3/100
1/1 [==============================] - 0s 799us/step - loss: 0.0070
Epoch 4/100
1/1 [==============================] - 0s 575us/step - loss: 0.0063
Epoch 5/100
1/1 [==============================] - 0s 535us/step - loss: 0.0057
Epoch 6/100
1/1 [==============================] - 0s 859us/step - loss: 0.0052
Epoch 7/100
1/1 [==============================] - 0s 776us/step - loss: 0.0048
Epoch 8/100
1/1 [==============================] - 0s 1ms/step - loss: 0.0045
Epoch 9/100
1/1 [==============================] - 0s 619us/step - loss: 0.0041
Epoch 10/100
1/1 [==============================] - 0s 1ms/step - loss: 0.0038
Epoch 11/100
1/1 [==============================] - 0s 627us/step - loss: 0.0035
Epoch 12/100
1/1 [==============================] - 0s 1ms/step - loss: 0.0032
Epoch 13/100
1/1 [=========================

We can now use the model for making predictions with the `predict()` method. The parameter `denormalized` will return values in the original scale of the data. In our case, US dollars.

In [14]:
M.predict(denormalized=True)

array([13520.626, 13333.145, 14279.886, 15165.678, 13989.352, 13865.876,
       13072.506], dtype=float32)

We now evaluate our model to inspect the statistics for the last epoch of training compared to a single test week.

In [15]:
M.evaluate()

{'mse': 0.0, 'rmse': 736.71, 'mape': 4.1}

Finally, we can now save the trained model on disk for later use.

In [16]:
M.save('/home/praveen/Desktop/Projects/technocolab/own/Deployement/bitcoin_model_prod_v0.h5')

Our `Model()` class can also load a previously trained model when instantiated with the `path` parameter.

In [17]:
M = Model(path='/home/praveen/Desktop/Projects/technocolab/own/Deployement/bitcoin_model_prod_v0.h5',
          data=model_data,
          variable='close',
          predicted_period_size=7)

In [18]:
M.predict(denormalized=True)

array([13520.626, 13333.145, 14279.886, 15165.678, 13989.352, 13865.876,
       13072.506], dtype=float32)

## New Data, Re-train Old Model
One strategy discussed earlier regards the re-training of our model with new data. In our case, our biggest concern is to shape data in a way that the model has been configured. As an example, we will configure our model to predict a week using 40 weeks. We will first train the model with the first 40 weeks of 2017, then continue to re-train it over the following weeks until we reach week 50.

First, let's build a model with the first set of data. Notice how we use `7*40 + 7` as the indexer. This is because we use 40 weeks for training and 1 week for testing. 

In [19]:
M1 = Model(data=model_data[0*7:7*40 + 7],
          variable='close',
          predicted_period_size=7)

In [20]:
M1.build()

In [21]:
M1.train()

In [24]:
#
#  Complete the range function and
#  the model_data filtering parameters
#  using an index to split the data in overlapping
#  groups of 7 days. Then, re-train our model
#  and collect the results.
#
#  The variables A, B, C, and D are placeholders.
#
A = 1
B = 10+1
 
results = []
for i in range(A, B):
    M1.train(model_data[i * 7:7*(40 + i) + 7])
    results.append(M.evaluate())

In [25]:
for i, result in enumerate(results):
    print(f'Week {40+i+1}: {result}')

Week 41: {'mse': 0.0, 'rmse': 736.71, 'mape': 4.1}
Week 42: {'mse': 0.0, 'rmse': 736.71, 'mape': 4.1}
Week 43: {'mse': 0.0, 'rmse': 736.71, 'mape': 4.1}
Week 44: {'mse': 0.0, 'rmse': 736.71, 'mape': 4.1}
Week 45: {'mse': 0.0, 'rmse': 736.71, 'mape': 4.1}
Week 46: {'mse': 0.0, 'rmse': 736.71, 'mape': 4.1}
Week 47: {'mse': 0.0, 'rmse': 736.71, 'mape': 4.1}
Week 48: {'mse': 0.0, 'rmse': 736.71, 'mape': 4.1}
Week 49: {'mse': 0.0, 'rmse': 736.71, 'mape': 4.1}
Week 50: {'mse': 0.0, 'rmse': 736.71, 'mape': 4.1}


In [26]:
M1.predict(denormalized=True)

array([14452.525, 14191.626, 13966.943, 12911.824, 12437.634, 10692.574,
       10949.453], dtype=float32)

## New Data, New Model
Another strategy is to create and train a new model evey time new data is available. This approach tends to reduce catastrophic forgetting, but training time increases as data increases. 

It's implementation is quite simple.

Let's assume we have old data for 49 weeks of 2017 and after a week we now have new data. We represent this wtih the variables `old_data` and `new_data`. 

In [27]:
old_data = model_data[0*7:7*48 + 7]

In [28]:
new_data = model_data[0*7:7*49 + 7]

In [29]:
M2 = Model(data=old_data,
          variable='close',
          predicted_period_size=7)

In [30]:
M2.build()
M1.train()

In [31]:
M2.predict(denormalized=True)

array([13749.089, 15275.757, 15755.756, 14448.42 , 14440.404, 14673.222,
       15833.19 ], dtype=float32)

Now, assume that new data is available. Using this technique we go ahead and create a new model using only the new data. 

In [34]:
#
#  Re-instantiate the model with the Model()
#  class using the new_data variable instead
#  of the old_data one. 
#
M3 = Model(data=new_data,
          variable='close',
          predicted_period_size=7)

In [35]:
M3.build()
M3.train()

In [36]:
M3.predict(denormalized=True)

array([17857.56 , 20057.475, 21191.371, 21276.479, 20818.354, 21753.63 ,
       22744.686], dtype=float32)

This approach is very simple to implement and tends to work well. We will be using this to deploy our application.